In [ ]:
!python -m spacy download en_core_web_md 
!python -m spacy download en_core_web_sm 
!pip install spacy
!pip install langdetect
!pip install dfply

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-15 03:09:57.438413: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42.8 MB 8.3 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-15 03:10:14.887215: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |███████████████████████████

In [ ]:
from spacy.lang.en import English
from spacy.symbols import ORTH
from spacy.lang.en.stop_words import STOP_WORDS
from dfply import *
from nltk import word_tokenize
from nltk.corpus import sentiwordnet as swn
from nltk import corpus
from nltk import FreqDist
from nltk.corpus import wordnet as wn
from nltk.corpus.reader import wordlist
from nltk.util import ngrams,bigrams,trigrams
from nltk.tokenize import word_tokenize
from langdetect import detect
from ast import literal_eval
import numpy as np
import pandas as pd
import os
import re
import glob
import string
import sys
import nltk
import spacy

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
stopwords = list(STOP_WORDS)
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nlp_sm = spacy.load("en_core_web_sm")
nlp_md = spacy.load("en_core_web_md")
config={"punct_chars":[";", '.', '...' ],"overwrite":True}
nlp_sm.add_pipe("sentencizer",config=config)
nlp_md.add_pipe("sentencizer",config=config)
punct = string.punctuation

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
stopwords = ['i','me','my','myself','we','our','ours','ourselves','you',"you're","you've","you'll","you'd",'your','yours',
 'yourself','yourselves','he','him','his','himself','she',"she's",'her','hers','herself','it',"it's",'its','itself','they',
 'them','their','theirs','themselves','what','which','who','whom','this','that',"that'll",'these','those','am','is','are','was',
 'were','be','been','being','have','has','had','having','do','does','did','doing','a','an','the','and','but','if','or','because',
 'as','until','while','of','at','by','for','with','about','between','into','through','to','from','in','out','over','under','again',
 'then','once','here','there','when','where','why','how','both','each','other','some','such','s','t', 'will',"’ll", "'ll","should","should've","'ve",'’ve',
 'now','d','m','o','re','ve','y','ma',"'d","'m",'’m',"'re",'’re',"'s",'’s','side','since',"’d","one","two","three","four","five","six","seven","eight","nine","ten","etc"]

In [ ]:
def detect_language(comment):
  try:
    return detect(comment)
  except:
    return "error"

def text_data_cleaning(sentence):
  if type(sentence)!= spacy.tokens.span.Span:
    doc = nlp_md(sentence)
  else:
    doc = sentence
  clean_tokens = []
  pos_tokens = []
  dep_tokens = []
  head_tokens = []
  for token in doc:
    if token.pos_ != "PRON" and token.text not in stopwords and token.text not in punct and not token.text.isnumeric(): 
      clean_tokens.append(token.lemma_.lower().strip())
      pos_tokens.append(token.pos_)
      dep_tokens.append(token.dep_)
      head_tokens.append(token.head.text.replace("'","\'"))
  return pd.Series([clean_tokens, pos_tokens, dep_tokens,head_tokens],index=["preprocess","pos","dep","head"])

def get_clausules(sentence):
  doc = nlp_md(sentence)
  sentences = []
  for sent in doc.sents:
    last = sent[0].i
    for tok in sent:
        is_conjuction = tok.pos_ == "CCONJ"#or  tok.pos_ == "CCONJ"
        if is_conjuction and len(doc) > tok.i+1 and doc[tok.i-1].pos_!= doc[tok.i+1].pos_:
          if len(doc[last:tok.i])>1:
            sentences.append(doc[last:tok.i])
          last = tok.i + 1
    if len(doc[last:sent[-1].i])>=1:
      sentences.append(doc[last:sent[-1].i])
  return sentences

def preprocesar_light(sentence):
  nounL = []
  doc = nlp_md(sentence.lower().strip())
  for n in doc:
    nounL.append(n.lemma_)
  return tuple(nounL)

def get_sentence_nounverb_aspect(unigram,bigram,trigram,aspects):
  fa = []#set()
  ngrams = unigram+bigram+trigram
  for t in ngrams:
    if t in aspects:
      fa.append(t)
  return fa

def get_aspect(sentence,aspect_lema):
  preprocess_light = preprocesar_light(sentence)
  unigram = list(ngrams(preprocess_light,1))
  bigram = list(ngrams(preprocess_light,2))
  trigram = list(ngrams(preprocess_light,3))
  everyaspect = set(aspect_lema)
  foundedaspect = get_sentence_nounverb_aspect(unigram,bigram,trigram,everyaspect)
  return foundedaspect

def get_specific_aspects(sentence):
    if sentence == 'alarm':
      return np.array('Alarm')
    elif sentence == 'mobile' or sentence == 'mobile app':
      return np.array('Mobile App')  
    elif sentence == 'screen':
      return np.array('Screen')    
    elif sentence == 'calorie':
      return np.array('Calories Burned')
    elif sentence == 'clock':
      return np.array('Clock Face')
    elif sentence == 'connect':
      return np.array('Connect')
    elif sentence == 'distance':
      return np.array('Distance Tracking')
    elif sentence == 'heart':
      return np.array('Heart Rate Tracking')
    elif sentence == 'notification':
      return np.array('Notifications')
    elif sentence == 'price':
      return np.array('Price')
    elif sentence == 'reminder':
      return np.array('Reminders')
    elif sentence == 'swim':
      return np.array('Swimproof + Swim tracking')
    elif sentence == 'sync':
      return np.array('Syncing')
    elif sentence == 'watch':
      return np.array('Watch Face')
    elif sentence == 'step':
      return np.array('Step Counter')
    elif sentence == 'battery':
      return np.array('Battery Life')
    elif sentence == 'sleep':
      return np.array('Sleep Tracker')
    elif sentence == 'app':
      return np.array('App')
    elif sentence == 'waterproof' or sentence == 'water resistance':
      return np.array('Waterproof')
    else: 
      return np.array('None')

In [ ]:
DATA = pd.read_excel('FINAL.xlsx',engine='openpyxl',sheet_name = "DATA")
NOUNS = pd.read_excel('NOUNS.xlsx',engine='openpyxl',sheet_name = "DATA") 
NOUNS["aspect"] = NOUNS.aspect.str.replace("\xa0", " ")
NOUNS["aspect_lema"] = NOUNS["aspect"].apply(preprocesar_light)

Module Pre-process

In [ ]:
df = DATA
df["CLAUSE"] = df["COMMENTS"].str.lower()
df["CLAUSE"] = df["CLAUSE"].apply(lambda x: re.sub("\\xa0", " ",x))
df["CLAUSE"] = df["CLAUSE"].apply(lambda x: re.sub("\.",". ",x))
df["CLAUSE"] = df["CLAUSE"].apply(lambda x: re.sub("\?+","? ",x))
df["CLAUSE"] = df["CLAUSE"].apply(lambda x: re.sub("\:+",": ",x))
df["CLAUSE"] = df["CLAUSE"].apply(lambda x: re.sub(";+","; ",x))
df["CLAUSE"] = df["CLAUSE"].apply(lambda x: re.sub(",+",", ",x))
df["CLAUSE"] = df["CLAUSE"].apply(lambda x: re.sub('\s+'," ",x))
df["CLAUSE"] = df["CLAUSE"].apply(lambda x: re.sub('[^a-zA-Z0-9\s]?hr[^a-zA-Z0-9\s]?',"hr",x))
df["LANGUAGE"]    = df["CLAUSE"].apply(detect_language)
df["CLAUSE"] = df["CLAUSE"].apply(get_clausules)
df = df.explode("CLAUSE").reset_index().rename(columns={"index":"comentario_id"})
df = df[~df["CLAUSE"].isna()].reset_index(drop=True)
df[["PREPROCESS","POS","DEP","HEAD"]]=df["CLAUSE"].apply(text_data_cleaning)
df["PREPROCESS_UNIGRAM"] = df["PREPROCESS"].apply(lambda x: list(ngrams(x,1)))
df["PREPROCESS_BIGRAM"] = df["PREPROCESS"].apply(lambda x: list(ngrams(x,2)))
df["PREPROCESS_TRIGRAM"] = df["PREPROCESS"].apply(lambda x: list(ngrams(x,3)))  
df["CLAUSE"] = df["CLAUSE"].astype(str)

In [ ]:
#print(df.loc[199].COMMENTS, df.loc[199].CLAUSE, df.loc[199].PREPROCESS, df.loc[199].ASPECT)

I was worried about all the reviews that mentioned endless vibrations, but so far no issues. I didnt realize I had my notifications like texts and calls turned off, so the only time it vibrated in the first few days of wearing it was when I hit 10k steps. I'd suggest reading up on how to set the correct settings for your goals and notifications  before saying all it does is vibrate. Using the app to learn how to use the fitbit is also a breeze. It's stylish and comfortable, even to wear to sleep. It definitely motivates me to get moving and the app is great for tracking your progress and customization. Was easy to set up and sync with my phone as well. Overall, I'm impressed and very happy with my purchase. i didnt realize i had my notifications like texts and calls turned off, so the only time it vibrated in the first few days of wearing it was when i hit 10k steps ['not', 'realize', 'notification', 'like', 'text', 'call', 'turn', 'off', 'so', 'only', 'time', 'vibrate', 'first', 'few'

Module Aspects Extraction and Filter

In [ ]:
FIL = []
for i in range(0,df.shape[0],1): FIL.append(get_aspect(df.CLAUSE[i],NOUNS.aspect_lema))
df["ASPECT"] = pd.Series(FIL)
df["LEN_TOK"] = df["ASPECT"].apply(len)
df.loc[df["LEN_TOK"] > 0,"ASPECT"] = [get_specific_aspects(item[0][0]) for item in df.loc[df["LEN_TOK"] > 0,"ASPECT"]]
df = df.reset_index().rename(columns={"index":"comentario_id"})
FIL = []
for i in np.arange(0, len(df), 1): FIL.append(sum([item.strip(string.punctuation).isalpha() for item in df.CLAUSE[i].split()]))
df['N_WORDS'] =  pd.Series(FIL)
df["ASPECT"] = df["ASPECT"].astype(str)
df["ASPECT"] = df["ASPECT"].replace('[]','None')
df = df >> mask((X.N_WORDS>=5) & (X.N_WORDS<=60), X.LANGUAGE == "en") >> select(~X.comentario_id, ~X.LANGUAGE, ~X.PREPROCESS, ~X.POS, ~X.DEP, ~X.HEAD, ~X.PREPROCESS_UNIGRAM, ~X.PREPROCESS_BIGRAM, ~X.PREPROCESS_TRIGRAM, ~X.LEN_TOK, ~X.N_WORDS, ~X.DEVICE_MODEL, ~X.TITLE, ~X.STATE, ~X.USER, ~X.DATE)
df1 = df >> mask(X.ASPECT != 'None')
df2 = df >> mask(X.ASPECT == 'None') >> sample(n = 1200 , replace=False)
final = pd.concat([df1, df2]) >> select(~X.comentario_id)

In [ ]:
final >> group_by('ASPECT') >> summarize(Cuenta = X.ASPECT.value_counts())

,ASPECT,Cuenta
0,Alarm,"Alarm 256 Name: ASPECT, dtype: int64"
1,App,"App 4154 Name: ASPECT, dtype: int64"
2,Battery Life,"Battery Life 3228 Name: ASPECT, dtype: int64"
3,Calories Burned,"Calories Burned 442 Name: ASPECT, dtype: int64"
4,Clock Face,"Clock Face 109 Name: ASPECT, dtype: int64"
5,Connect,"Connect 835 Name: ASPECT, dtype: int64"
6,Distance Tracking,"Distance Tracking 250 Name: ASPECT, dtype: ..."
7,Heart Rate Tracking,"Heart Rate Tracking 2351 Name: ASPECT, dtyp..."
8,Mobile App,"Mobile App 323 Name: ASPECT, dtype: int64"
9,None,"None 1200 Name: ASPECT, dtype: int64"


In [ ]:
final.to_excel('AMAZON_FINAL.xlsx', sheet_name = "DATA", index = None, header=True)